In [1]:
import numpy as np
from sympy import *

# Modeling an Insect's wing

**Background:**

Wing is modeled as a cantilever beam with distributed load and a joint where the material property changes.

Deflection dependent on:

    -length of beam
    -load
    -EI (Young's modulus, wing's second moment of area) (bending stiffness)





**Purpose:**  

The purpose is to model an insect's wing that has minimized stress while allowing for some deformation as it has been shown that a flexible wing has beneficial aspects to it like reducing energy consumption and increasing lift forces. 

Trying to choose a model that we think would make an efficient micro aerial vehicle 

Also for the property materials, use resilin/chitin inspired from insect's biology 

minimizing stress/keeping it under a threshold while allowing for a reasonable deformation that is beneficial

--discuss what the best model + bending stiffess constants (or varying EI(x)) could be for an efficient micro aerial vehicle 




**Assumptions made:**
   * uniform mass distribution
   * isotropic material 
   * inertial forces are roughly equal in magnitude to aeordynamic forces (air load)
   * ignoring torsion
    

Constants:

In [9]:
wingLength = 6
jointStartLocation = 3
jointSize = 0.5
wingRadius = 2
angleOfWing = 0.2
height = 1 
weight = 48 #units = mg 
load = 2
young = 1 
j_young = 5
air_load = 5
wingWeight = 0.8

In [3]:
def wingSim(wingWeight, young, j_young, air_load, wingLength, jointStartLocation, jointSize, wingRadius, angleOfWing):
    daMoment = getSecondMomemnt(wingRadius)
    wDistrib = wingWeight/ wingLength
    totalD = wDistrib + air_load
    #incorporating the angle change
    trueTotalD = totalD * cos(angleOfWing)
    themSupports = getSupportReactions(trueTotalD, wingLength)
    listODeflections = getDeflection(young, j_young, trueTotalD, wingLength, jointStartLocation, jointSize, daMoment)
    

In [4]:
def getSupportReactions(totalD, wingLength):
    x, y = symbols('x'), symbols('y')
    #since there are no forces in the x-direction, the support in the d direction would be 0
    xSup = 0
    ySup = integrate(totalD, (x, 0, wingLength)) 

    #we can integrate the total distribution over the length of the wing to find the moment
    mSup = integrate(totalD * x, (x, 0, wingLength))
    return [xSup, ySup, mSup]

Calculating the second moment of area - I:

Assumption: circular cross section 


In [5]:
def getSecondMomemnt(wingRadius):
    return (pi/4 ) * (wingRadius ** 4)

In [6]:
def getDeflection(young, j_young, trueTotalD, wingLength, jointStartLocation, jointSize, daMoment):
    
    x, y = symbols('x'), symbols('y')
    #Calculating Moment. Suppose cut at x. Only one cut necessary since the moment doesn't depend on material properties.
    momentExpr = -trueTotalD * ((wingLength - x)**2)*(1/2)
    print(momentExpr)

    #between x = 0 to x = jointStartLocation (assuming JointStartLocation < wingLength)
    deflectionExpr1 = calculateDeflection(young, momentExpr, daMoment)
    #both constants of integration = 0 since at x = 0, v(x) = 0 and dv/dx = 0

    #at the joint (between jointStartLocation and (jointStartLocation + jointLength))
    draftdeflectionExpr2 = calculateDeflection(j_young, momentExpr, daMoment)
    
    #at x = jointStartLocation, d(deflectionExpr1)/dx = d(deflectionExpr2), so the first constant is them subtracted from each other
    c1 = diff(deflectionExpr1, x).subs(x, jointStartLocation) - diff(draftdeflectionExpr2, x).subs(x, jointStartLocation)
    draftdeflectionExpr2 += c1 * x
    
    #at x = jointStartLocation, deflectionExpr1 = deflectionExpr2, so the second constant is them subtracted from each other
    c2 = deflectionExpr1.subs(x, jointStartLocation) - draftdeflectionExpr2.subs(x, jointStartLocation)
    deflectionExpr2 = draftdeflectionExpr2 + c2

    #calcuting deflection between (jointStartLocation + jointLength) and the end (wingLength)
    draftdeflectionExpr3 = calculateDeflection(young, momentExpr, daMoment)
    #at x = jointStartLocation + jointSize, d(deflectionExpr1)/dx = d(deflectionExpr2), so the third constant is them subtracted from each other
    c3 = diff(deflectionExpr2, x).subs(x, (jointStartLocation + jointSize)) - diff(draftdeflectionExpr3, x).subs(x, (jointStartLocation + jointSize))
    draftdeflectionExpr2 += c3 * x
    
    #at x = jointStartLocation + jointSize, deflectionExpr1 = deflectionExpr2, so the fourth constant is them subtracted from each other
    c4 = deflectionExpr2.subs(x, (jointStartLocation + jointSize)) - draftdeflectionExpr3.subs(x, (jointStartLocation + jointSize))
    deflectionExpr3 = draftdeflectionExpr3 + c4
    
    print(deflectionExpr1)
    print(deflectionExpr2)
    print(deflectionExpr3)
    return [deflectionExpr1, deflectionExpr2, deflectionExpr3]

In [7]:
def calculateDeflection(young, momentExpression, daSecondMoment):
    #integrate twice since EI * d^2(v)/dx^2 = M(x)     
    return (1/(young * daSecondMoment)) * integrate(integrate(momentExpression))

In [8]:
wingSim(wingWeight, young, j_young, air_load, wingLength, jointStartLocation, jointSize, wingRadius, angleOfWing)

-2.51550421645919*(6 - x)**2
(-0.209625351371599*x**4 + 5.03100843291837*x**3 - 45.2790758962654*x**2)/(4*pi)
-31.6953531273858*x/pi + (-0.209625351371599*x**4 + 5.03100843291837*x**3 - 45.2790758962654*x**2)/(20*pi) + 37.3552376144189/pi
(-0.209625351371599*x**4 + 5.03100843291837*x**3 - 45.2790758962654*x**2)/(4*pi) + 0.505721160183995/pi
